In [12]:
import model_classes.RNN_model_class as RNN_model_class
import yaml
import main

yaml_path = 'train_params.yaml'
with open(yaml_path, 'r') as file:
    params = yaml.safe_load(file)

params['data_params']['ticker'] = 'AAPL'
model = None
scaler_X = None
scaler_Y = None
    
RNN_model = RNN_model_class.RNN_model(params, model, scaler_X, scaler_Y)
model, scaler_X, scaler_Y, params = RNN_model.train_model()
main.save_training(model, scaler_X, scaler_Y, params)

DEBUG:yfinance:Entering download()
DEBUG:yfinance:Disabling multithreading because DEBUG logging enabled
DEBUG:yfinance: Entering history()
DEBUG:peewee:('SELECT "t1"."key", "t1"."value" FROM "_kv" AS "t1" WHERE ("t1"."key" = ?) LIMIT ? OFFSET ?', ['AAPL', 1, 0])
DEBUG:yfinance:  Entering history()
DEBUG:yfinance:AAPL: Yahoo GET parameters: {'period1': '2024-08-02 20:20:27-04:00', 'period2': '2024-08-05 20:20:27-04:00', 'interval': '1m', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
DEBUG:yfinance:   Entering get()
DEBUG:yfinance:url=https://query2.finance.yahoo.com/v8/finance/chart/AAPL
DEBUG:yfinance:params={'period1': 1722644427, 'period2': 1722903627, 'interval': '1m', 'includePrePost': False, 'events': 'div,splits,capitalGains'}
DEBUG:yfinance:    Entering _get_cookie_and_crumb()
DEBUG:yfinance:cookie_mode = 'basic'
DEBUG:yfinance:     Entering _get_cookie_and_crumb_basic()
DEBUG:yfinance:reusing cookie
DEBUG:yfinance:reusing crumb
DEBUG:yfinance:     Exiting _get_

Epoch [10/100], Train_Loss: 0.0389
Epoch [10/100], Test_Loss: 0.0856
Epoch [20/100], Train_Loss: 0.0127
Epoch [20/100], Test_Loss: 0.0147
Epoch [30/100], Train_Loss: 0.0021
Epoch [30/100], Test_Loss: 0.0020
Epoch [40/100], Train_Loss: 0.0033
Epoch [40/100], Test_Loss: 0.0019
Epoch [50/100], Train_Loss: 0.0033
Epoch [50/100], Test_Loss: 0.0008
Epoch [60/100], Train_Loss: 0.0026
Epoch [60/100], Test_Loss: 0.0021
Epoch [70/100], Train_Loss: 0.0022
Epoch [70/100], Test_Loss: 0.0007
Epoch [80/100], Train_Loss: 0.0021
Epoch [80/100], Test_Loss: 0.0013
Epoch [90/100], Train_Loss: 0.0020
Epoch [90/100], Test_Loss: 0.0008
Epoch [100/100], Train_Loss: 0.0020
Epoch [100/100], Test_Loss: 0.0010


In [10]:
import os
import shutil
from pathlib import Path

def delete_folder(folder_path):
    if os.path.isdir(folder_path):
        try:
            shutil.rmtree(folder_path)
            print(f"Successfully deleted the folder and all its contents: {folder_path}")
            success = True
        except FileNotFoundError:
            print(f"Folder not found: {folder_path}")
            success = False
        except PermissionError:
            print(f"Permission denied: {folder_path}")
            success = False
        except OSError as e:
            print(f"Error: {e.strerror}")
            success = False
    else:
        print(f"The path is not a directory: {folder_path}")
        success = False
    return success



def manage_folders(base_dir):
    
    base_path = Path(base_dir)
    # Get a list of all directories in the base directory
    folders = [folder for folder in base_path.iterdir() if folder.is_dir()]
    
    # Check if the number of folders exceeds 10
    if len(folders) >= 2:
        print(f"Number of folders exceeds 10. Total folders: {len(folders)}")
        for folder in folders:
            delete_folder(folder)  # Delete each folder
        print("All folders have been deleted.")
    else:
        print(f"Number of folders is within limit. Total folders: {len(folders)}")


#set baser_dir to current path 
MODEL_DB_DIR = r'C:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\Model_DB'



print(MODEL_DB_DIR)
manage_folders(MODEL_DB_DIR)


C:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\Model_DB
Number of folders exceeds 10. Total folders: 2
Successfully deleted the folder and all its contents: C:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\Model_DB\AAPL_RNN_19.02
Successfully deleted the folder and all its contents: C:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\Model_DB\AAPL_RNN_19.27
All folders have been deleted.


In [2]:
# define global variable for older predictions 
all_preds = {'Datetime': [], 'Prediction': [], 'Model_name': []}

In [15]:
import pandas as pd 
import helper_fct
import model_classes.RNN_model_class as RNN_model_class
from sklearn.metrics import mean_absolute_error as MAE
def predict_model( data, model_name):
    # set folder dir for model
    model = RNN_model_class.RNN_model()
    model.load_model(model_name=model_name,model_db=MODEL_DB_DIR)
    prediction = model.predict(data)

    return prediction
MODEL_DB_DIR = r'C:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\Model_DB'
ticker  = 'AAPL'
model_list = ['AAPL_RNN_20.06','AAPL_RNN_20.23']
mse_dict = {}

# load prediction data 
data = helper_fct.get_predict_data(ticker=ticker, interval='1m', seq_length=60)
#print(data)
plot_data = data['Close'].reset_index()

# refactor datetime to the same timezone for merging 
plot_data['Datetime'] = pd.to_datetime(plot_data['Datetime'], utc=True)
plot_data['Datetime'] = plot_data['Datetime'].dt.tz_convert('America/New_York')

for model_name in model_list :
    # predict with the model
    prediction = predict_model(data=data, model_name=model_name)

    all_preds['Datetime'].append(prediction['Datetime'])
    all_preds['Prediction'].append(prediction['Prediction'])
    all_preds['Model_name'].append(model_name)

        
# refactor df so that it has the structure:
# Datetime      Model_name_A            Model_name_B              Close
# date          prediction_model A      prediction_model B        actual_price
df_plot = pd.DataFrame(all_preds)

# refactor datetime to the same timezone for merging 
df_plot['Datetime'] = pd.to_datetime(df_plot['Datetime'], utc=True)
df_plot['Datetime'] = df_plot['Datetime'].dt.tz_convert('America/New_York')

# merging both dataframes 
df_plot = df_plot.drop_duplicates(subset=['Datetime', 'Model_name'])               
pivoted_df = df_plot.pivot(index='Datetime', columns='Model_name', values='Prediction').reset_index()
plot_data = pd.merge(pivoted_df, plot_data, on="Datetime", how='outer').sort_values(by='Datetime').reset_index(drop=True)
print(plot_data)
# calculate MSE
df_mse = plot_data.dropna()
for model_name in model_list :
    if df_mse.empty:
        mse_dict[model_name] = None
            
    else :
        mse_dict[model_name] = MAE(df_mse['Close'],df_mse[model_name])

print(df_mse)
print(mse_dict)

                    Datetime  AAPL_RNN_20.06  AAPL_RNN_20.23       Close
0  2024-08-05 15:00:00-04:00             NaN             NaN  204.619995
1  2024-08-05 15:01:00-04:00             NaN             NaN  204.095001
2  2024-08-05 15:02:00-04:00             NaN             NaN  205.331604
3  2024-08-05 15:03:00-04:00             NaN             NaN  205.919998
4  2024-08-05 15:04:00-04:00             NaN             NaN  206.350006
..                       ...             ...             ...         ...
57 2024-08-05 15:57:00-04:00             NaN             NaN  208.610001
58 2024-08-05 15:58:00-04:00             NaN             NaN  208.940002
59 2024-08-05 15:59:00-04:00             NaN             NaN  209.190002
60 2024-08-05 16:01:00-04:00      204.487457             NaN         NaN
61 2024-08-05 16:02:00-04:00             NaN      214.159973         NaN

[62 rows x 4 columns]
Empty DataFrame
Columns: [Datetime, AAPL_RNN_20.06, AAPL_RNN_20.23, Close]
Index: []
{'AAPL_RNN_20.06

c:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\app\model_classes\RNN_model_class.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[features.columns] = self.scaler_X.transform(features[features.columns])
c:\Users\phili\OneDrive\Projekte\Code\GitHub\timeseries_forcast_in_finance_MLops\app\model_classes\RNN_model_class.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features[features.columns] = self.scaler_X.transform(features[features.columns])


In [ ]:
import main 

   df_mse = plot_data.dropna()
    for model_name in model_list :
        if df_mse.empty:
            mae_dict[model_name] = None
            
        else :
            mae_dict[model_name] = MAE(df_mse['Close'],df_mse[model_name])